In [1]:
MILVUS_HOST="milvus-service"
MILVUS_PORT="19530"

LLAMA_HOST="llama-service"
LLAMA_PORT="8080"

In [2]:
from pymilvus import connections, utility

# Connect to Milvus Database
connections.connect(host="milvus-service", port="19530")

In [3]:
from pymilvus import connections, utility
from langchain_community.llms import LlamaCpp
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
import requests
import os
FNAME = "granite-3.0-8b-instruct-Q4_K_M.gguf"

if not os.path.exists(FNAME):
    res = requests.get('https://huggingface.co/bartowski/granite-3.0-8b-instruct-GGUF/resolve/main/granite-3.0-8b-instruct-Q4_K_M.gguf')
    with open(FNAME, 'wb') as file:
        file.write(res.content)

In [5]:
llm = LlamaCpp(
    model_path="granite-3.0-8b-instruct-Q4_K_M.gguf", 
    temperature=0,
    min_tokens=5,
    max_tokens=250,
    n_ctx=0,
    stop=["Human:", "Observation"])

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3517: UserWarning: WARNING! min_tokens is not default parameter.
                min_tokens was transferred to model_kwargs.
                Please confirm that min_tokens is what you intended.
  if await self.run_code(code, result, async_=asy):
llama_model_loader: loaded meta data with 40 key-value pairs and 362 tensors from granite-3.0-8b-instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = granite
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Granite 3.0 8b Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader

In [6]:
template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

In [7]:
agent = prompt | llm

In [8]:
agent.invoke({"query": "What sport is played at the US Open?"})

llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    30 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6988.05 ms /    45 tokens


'\n\nThe sport played at the US Open is tennis.'

In [9]:
agent.invoke({"query": "Where was the 2024 US Open Tennis Championship?"})

Llama.generate: 2 prefix-match hit, remaining 36 prompt tokens to eval
llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    36 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    57 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20629.29 ms /    93 tokens


'\n\nThe 2024 US Open Tennis Championship has not been announced yet, so the location is not known. Therefore, I do not know where the 2024 US Open Tennis Championship will be held.'

In [10]:
urls = [
    "https://www.ibm.com/case-studies/us-open",
    "https://www.ibm.com/sports/usopen",
    "https://newsroom.ibm.com/US-Open-AI-Tennis-Fan-Engagement",
    "https://newsroom.ibm.com/2024-08-15-ibm-and-the-usta-serve-up-new-and-enhanced-generative-ai-features-for-2024-us-open-digital-platforms",
]

In [11]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[0]

Document(metadata={'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM', 'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI models with watsonx.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nU.S. Open | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nCase Studies\n\n\n\nUS Open \n\n\n\n\n                \n\n\n\n  \n    Acing the US Open digital experience\n\n\n\n\n\n\n    \n\n\n            \n\n                    \n\n\n  \n  \n      AI models built with watsonx transform data into insight\n  \n\n\n\n\n    \n\n\n                \n\n\nGet the latest AI and tech insights\n\n\n\nLearn More\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFor two weeks at the end of summer, nearly one million people make the journey to Flushing, New York, to wat

In [12]:
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipykernel_43/3401734470.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
vector_store = Milvus.from_documents(
            doc_splits,
            embedding=embeddings,
            collection_name="agentic_rag",
            connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT}
        )

In [16]:
retriever = vector_store.as_retriever()

In [29]:
@tool
def get_IBM_US_Open_context(question: str):
    """Get context about IBM's involvement in the 2024 US Open Tennis Championship."""
    context = retriever.invoke(question)
    return context

In [30]:
tools = [get_IBM_US_Open_context]

In [44]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [45]:
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

In [46]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

In [47]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.System

In [48]:
prompt = prompt.partial(
            tools=render_text_description_and_args(list(tools)),
            tool_names=", ".join([t.name for t in tools]),
        )

In [49]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [50]:
memory = ConversationBufferMemory()

In [51]:
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory
)

In [52]:
agent_executor.invoke({"input": "Where was the 2024 US Open Tennis Championship?"})



> Entering new AgentExecutor chain...


llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   327 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    99 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   65692.77 ms /   426 tokens
Llama.generate: 317 prefix-match hit, remaining 719 prompt tokens to eval



Thought: The human asked about the location of the 2024 US Open Tennis Championship. I need to find out where it will be held.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": {
    "question": "Where will the 2024 US Open Tennis Championship be held?"
  }
}
```
[Document(metadata={'source': 'https://newsroom.ibm.com/US-Open-AI-Tennis-Fan-Engagement', 'title': 'US Open: IBM and AI Can Help Engage Tennis Fans ', 'description': '', 'language': 'en-us', 'pk': 455129936176156242}, page_content='As the US Open gets underway in New York this week, the world’s top tennis players are performing in front of empty seats—not the 800,000 or so fans who would normally fill the Billie Jean King National Tennis Center during the two-week tournament.'), Document(metadata={'source': 'https://www.ibm.com/case-studies/us-open', 'title': 'U.S. Open | IBM', 'description': 'To help the US Open stay on the cutting edge of customer experience, IBM Consulting built powerful generative AI

llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   719 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    61 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  147063.61 ms /   780 tokens



Action:
```
{
  "action": "Final Answer",
  "action_input": {
    "answer": "The 2024 US Open Tennis Championship will be held in Flushing, Queens, New York."
  }
}
```


ValidationError: 2 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value={'answer': 'The 2024 US O...ing, Queens, New York.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value={'answer': 'The 2024 US O...ing, Queens, New York.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type

In [54]:
agent_executor.invoke(
    {"input": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"}
)



> Entering new AgentExecutor chain...


Llama.generate: 305 prefix-match hit, remaining 49 prompt tokens to eval
llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    49 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    99 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   31178.98 ms /   148 tokens
Llama.generate: 344 prefix-match hit, remaining 1000 prompt tokens to eval



AI: Thought: I need to find out how IBM used watsonx at the 2024 US Open Tennis Championship.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": {
    "question": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"
  }
}
```
[Document(metadata={'source': 'https://newsroom.ibm.com/2024-08-15-ibm-and-the-usta-serve-up-new-and-enhanced-generative-ai-features-for-2024-us-open-digital-platforms', 'title': 'IBM and the USTA Serve Up New and Enhanced Generative AI Features for 2024 US Open Digital Platforms', 'description': "IBM and the United States Tennis Association (USTA) announced several watsonx-powered fan features coming to the US Open digital platforms ahead of this year's tournament. These new and enhanced capabilities – a product of collaboration between IBM and the USTA digital team – aim to deliver a more informative and engaging experience for millions of tennis fans around the world.", 'language': 'en-us', 'pk': 455129936176156303}, page_cont

llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1000 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  292026.20 ms /  1100 tokens



Action:
```
{
  "action": "Final Answer",
  "action_input": {
    "answer": "IBM used watsonx at the 2024 US Open Tennis Championship to power new and enhanced fan features on the digital platforms. These features were developed by IBM in collaboration with the USTA digital team to deliver a more informative and engaging experience for tennis fans around the world."
  }
}
```


ValidationError: 2 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value={'answer': 'IBM used wats...fans around the world.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value={'answer': 'IBM used wats...fans around the world.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type

In [53]:
agent_executor.invoke({"input": "What is the capital of France?"})



> Entering new AgentExecutor chain...


Llama.generate: 297 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =    4380.59 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    19 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    62 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17705.46 ms /    81 tokens



Assistant:
Thought: The user is asking for the capital of France. I don't need to use any tools for this information.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The capital of France is Paris."
}


> Finished chain.


{'input': 'What is the capital of France?',
 'history': '',
 'output': 'The capital of France is Paris.'}